## Transaction: 

In MySQL, transactions allow you to execute a set of SQL queries as a single unit of work. If any query fails, you can roll back all the changes made by previous queries in the transaction. If all queries are successful, you can commit the changes, making them permanent.

### Key Concepts:
1. **START TRANSACTION**: Begins a new transaction.
2. **COMMIT**: Saves the changes made in the transaction.
3. **ROLLBACK**: Undoes the changes made in the transaction if something goes wrong.

#### Example of Using Transactions

Let's assume we have two tables: `accounts` and `transactions`.

```sql
CREATE TABLE accounts (
    account_id INT PRIMARY KEY,
    balance DECIMAL(10, 2)
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    from_account INT,
    to_account INT,
    amount DECIMAL(10, 2),
    timestamp DATETIME
);
```

### Example 1: Simple Transaction for Transferring Money Between Two Accounts

This example transfers $100 from account 1 to account 2. If any part of the transaction fails (e.g., insufficient balance), the whole transaction is rolled back.

```sql
-- Start the transaction
START TRANSACTION;

-- Check if the source account has enough balance
SELECT balance FROM accounts WHERE account_id = 1;

-- Assuming account 1 has enough balance, deduct $100 from account 1
UPDATE accounts
SET balance = balance - 100
WHERE account_id = 1;

-- Add $100 to account 2
UPDATE accounts
SET balance = balance + 100
WHERE account_id = 2;

-- Insert the transaction record
INSERT INTO transactions (from_account, to_account, amount, timestamp)
VALUES (1, 2, 100, NOW());

-- If all queries succeed, commit the transaction
COMMIT;
```

If something goes wrong during the transaction, you can call `ROLLBACK` instead of `COMMIT`:

```sql
-- Roll back the transaction if an error occurs
ROLLBACK;
```

### Example 2: Handling Errors in Transactions

Suppose we want to handle the case where account 1 doesn't have enough balance. We can add a conditional check:

```sql
-- Start the transaction
START TRANSACTION;

-- Check the balance of account 1
SELECT balance INTO @balance FROM accounts WHERE account_id = 1;

-- If the balance is sufficient, proceed with the transfer
IF @balance >= 100 THEN
    -- Deduct $100 from account 1
    UPDATE accounts
    SET balance = balance - 100
    WHERE account_id = 1;
    
    -- Add $100 to account 2
    UPDATE accounts
    SET balance = balance + 100
    WHERE account_id = 2;

    -- Record the transaction
    INSERT INTO transactions (from_account, to_account, amount, timestamp)
    VALUES (1, 2, 100, NOW());
    
    -- Commit the changes
    COMMIT;
ELSE
    -- If there isn't enough balance, roll back the changes
    ROLLBACK;
END IF;
```

This example ensures that if account 1 doesn't have enough balance, the transaction is rolled back.

### Example 3: Using `SAVEPOINT`

A savepoint allows you to roll back part of a transaction without rolling back the entire transaction. Here’s how to use it:

```sql
-- Start the transaction
START TRANSACTION;

-- Set a savepoint
SAVEPOINT before_transfer;

-- Try to deduct $100 from account 1
UPDATE accounts
SET balance = balance - 100
WHERE account_id = 1;

-- Check if the deduction was successful (imagine some condition here)
-- If not successful, rollback to the savepoint
ROLLBACK TO before_transfer;

-- If all goes well, commit the changes
COMMIT;
```

With `SAVEPOINT`, you can roll back to specific stages within the transaction.

These examples show how transactions can be useful for ensuring data integrity when working with multiple related queries.

### Remarks: 

In MySQL, you don't need to explicitly declare user-defined variables like `@balance` before using them. MySQL automatically initializes a variable when you first assign a value to it using the `SELECT INTO` syntax or through direct assignment.

In Example 2 (`Handling Errors in Transactions`), when you run this query:

```sql
SELECT balance INTO @balance FROM accounts WHERE account_id = 1;
```

MySQL will create the `@balance` variable and assign it the value from the `balance` column for `account_id = 1`.

### Important Points to Remember:
- User-defined variables (those starting with `@`) are session-specific and exist as long as the session is active.
- You don't need to declare or specify their data type.
- If no rows match the `SELECT` query, `@balance` will be set to `NULL`.

So, in your case, there is no need to declare `@balance` before using it. MySQL handles it automatically when you assign a value to it with `SELECT INTO`.

### Example with Error Handling
Here’s the modified query where `@balance` is used without an explicit declaration:
DELIMITER //

CREATE PROCEDURE transfer_funds()
BEGIN
    DECLARE balance DECIMAL(10,2);

    -- Start the transaction
    START TRANSACTION;

    -- Fetch the balance of account 1
    SELECT balance INTO @balance FROM accounts WHERE account_id = 1;

    -- Check if the balance is sufficient
    IF @balance >= 100 THEN
        -- Deduct $100 from account 1
        UPDATE accounts
        SET balance = balance - 100
        WHERE account_id = 1;

        -- Add $100 to account 2
        UPDATE accounts
        SET balance = balance + 100
        WHERE account_id = 2;

        -- Record the transaction
        INSERT INTO transactions (from_account, to_account, amount, timestamp)
        VALUES (1, 2, 100, NOW());

        -- Optionally, delete some temporary data (example)
        DELETE FROM temp_transactions WHERE account_id = 1;

        -- Commit the transaction
        COMMIT;
    ELSE
        -- If balance is insufficient, roll back the transaction
        ROLLBACK;
    END IF;
END //

DELIMITER ;


The key takeaway is that MySQL allows the implicit use of variables without prior declaration.

When using **user-defined session variables** (variables that begin with `@`) in MySQL **outside of stored procedures or functions**, you do not need to declare them, as mentioned earlier. However, **inside stored procedures or functions**, it is generally better to use **local variables** (without the `@` symbol), and these must be explicitly declared using the `DECLARE` statement.

### Key Differences:
1. **User-defined session variables** (prefixed with `@`) exist throughout the session and do not need explicit declaration.
2. **Local variables** (declared using `DECLARE`) are scoped within the stored procedure or function and must be declared before use.

### Example Without Declaring (`@` Session Variables):
You can use session variables like `@balance` in a stored procedure, and you wouldn't need to declare them explicitly. Here's how it would look:

```sql
DELIMITER $$

CREATE PROCEDURE transfer_money()
BEGIN
    START TRANSACTION;
    
    -- No need to declare @balance, it’s a session variable
    SELECT balance INTO @balance FROM accounts WHERE account_id = 1;
    
    IF @balance >= 100 THEN
        -- Deduct $100 from account 1
        UPDATE accounts
        SET balance = balance - 100
        WHERE account_id = 1;
        
        -- Add $100 to account 2
        UPDATE accounts
        SET balance = balance + 100
        WHERE account_id = 2;
        
        -- Record the transaction
        INSERT INTO transactions (from_account, to_account, amount, timestamp)
        VALUES (1, 2, 100, NOW());
        
        -- Commit the transaction
        COMMIT;
    ELSE
        -- If insufficient balance, roll back the transaction
        ROLLBACK;
    END IF;
    
END$$

DELIMITER ;
```

### Using Local Variables (Recommended for Procedures/Functions):
In stored procedures or functions, it is **best practice** to use local variables, which must be explicitly declared using `DECLARE`. Local variables are scoped only to the procedure or function and don’t persist after execution.

Here’s an example using a **local variable**:

```sql
DELIMITER $$

CREATE PROCEDURE transfer_money_local()
BEGIN
    -- Declare local variable
    DECLARE balance DECIMAL(10, 2);

    START TRANSACTION;
    
    -- Fetch the balance from account 1 into the local variable
    SELECT balance INTO balance FROM accounts WHERE account_id = 1;
    
    IF balance >= 100 THEN
        -- Deduct $100 from account 1
        UPDATE accounts
        SET balance = balance - 100
        WHERE account_id = 1;
        
        -- Add $100 to account 2
        UPDATE accounts
        SET balance = balance + 100
        WHERE account_id = 2;
        
        -- Record the transaction
        INSERT INTO transactions (from_account, to_account, amount, timestamp)
        VALUES (1, 2, 100, NOW());
        
        -- Commit the transaction
        COMMIT;
    ELSE
        -- If insufficient balance, roll back the transaction
        ROLLBACK;
    END IF;
    
END$$

DELIMITER ;
```

### Key Points:
- **Session variables** (like `@balance`) don't need explicit declaration and are available throughout the session, but they persist across stored procedures/functions and should be used with caution inside them.
- **Local variables** (declared with `DECLARE`) are recommended for use inside stored procedures/functions because they are scoped locally and do not persist beyond the procedure or function execution.

In summary, while you don't have to declare `@` variables even inside stored procedures, using `DECLARE` to create local variables is considered a cleaner, more controlled practice in procedures and functions.